In [ ]:
from pinpoint import Tracker, VideoReader
import matplotlib.pyplot as plt
import time
import h5py
import glob
import numpy as np
import cv2

In [ ]:
videos = glob.glob("/media/jake/*.mp4")
videos

In [ ]:
video = VideoReader(videos[0])

In [ ]:
frame = video.read()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(frame)
plt.show()

In [ ]:
from pinpoint.utils import grayscale, adaptive_threshold, find_contours, find_candidate_barcodes, fit_polygon

In [ ]:
# modify these parameters to adjust the threshold
block_size = 1001
offset = -20
resize = 3

resized = cv2.resize(frame, (0,0), None, fx = resize, fy = resize)
gray = grayscale(resized, 'green')
thresh = adaptive_threshold(gray, block_size, offset)

plt.figure(figsize=(50,50))
plt.imshow(gray, cmap='gray')
plt.show()

plt.figure(figsize=(50,50))
plt.imshow(thresh, cmap='gray')
plt.show()

In [ ]:
contours = find_contours(thresh)
areas = np.array(map(cv2.contourArea, contours))
len(contours)

plt.hist(np.sqrt(areas[areas>10]), bins=50)
plt.xscale('log')
plt.show()

In [ ]:
tracker = Tracker(source=videos[0],
                  block_size=block_size,
                  offset=offset,
                  distance_threshold=20,
                  area_range=(50,500),
                  tolerance=0.1,
                  channel='green',
                  resize=resize)

In [ ]:
# set your barcode dictionary
tracker.load_dict('/home/jake/pinpoint/barcodes/5x5_7bit/master_list.pkl')

In [ ]:
t0 = time.time()
tracker.track('output.h5',batch_size=200, n_jobs=-1)
t1 = time.time()
dt = t1-t0
fps = tracker.total_frames/dt
print(fps)

In [ ]:
f = h5py.File('output.h5', 'r')

frame_idx = f['data/frame_idx'].value
corners = f['data/corners'].value
identity = f['data/identity'].value
distances = f['data/distances'].value

fps = f.attrs['fps']
codec = f.attrs['codec']
height = f.attrs['height']
width = f.attrs['width']
total_frames = f.attrs['total_frames']

f.close()

In [ ]:
np.unique(identity[distances == 0])

In [ ]:
plt.hist(distances, bins=20)
plt.show()